In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
import sys
import os

In [ ]:
# Read data from fits table file
filepath = '../../Data/CoNFIG/asu.fit'
hdul = fits.open(filepath)
data = hdul[1].data
hdul.close()

In [9]:
# Read data from '|' seperated file
#filepath = '../../Data/CoNFIG/asu.tsv'
#filepath = '../../Data/FRICAT/fr1.tsv'
#filepath = '../../Data/FRIICAT/asu.tsv'
#filepath = '../../Data/FRICAT/sfr1.tsv'
#filepath = '../../Data/FR0CAT/asu.tsv'
filepath = '../../Data/Garon 2017/matched.tsv'

data = pd.read_csv(filepath, sep='|', header=None, skiprows=3)
data = np.array(data)
#data = data[3623:,:] # some data downloaded with arc_size=2.5... so redownload that data

coords = data[:,0]
ra, dec = np.empty_like(coords), np.empty_like(coords)
# for i, coord in enumerate(coords):
#     ra[i] = coord[1:3]+' '+coord[3:5]+' '+coord[5:10]
#     dec[i] = coord[10:13]+' '+coord[13:15]+' '+coord[15:]
for i, coord in enumerate(coords):
    ra[i] = coord[1:3]+' '+coord[3:5]+' '+coord[5:9]
    dec[i] = coord[9:12]+' '+coord[12:14]+' '+coord[14:]
# ra, dec = data[:,0], data[:,1]
# ra, dec = data[:,3], data[:,4]
ra[3622], dec[3622] # missing ra and dec from first cutout server

('22 17 32.3', '+09 49 20')

In [6]:
# Download FITS image files from FIRST cutout server
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException

download_path = r'C:\Users\JakeM\Google Drive\University of Birmingham\Year 4\Project\Data\Garon 2017\FIRST-matched'
#download_path = r'C:\Users\JakeM\Downloads'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_path}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

# Get webpage that cutouts can be downloaded from
driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")

size_arcmin = 4.5
imagesize = driver.find_element_by_name('ImageSize')
driver.execute_script("arguments[0].value=arguments[1]",imagesize,'{}'.format(size_arcmin))

def update_progress(process, progress):
    # Simple progress bar for long loops (e.g. data loading/augmentation)
    block = int(round(10*progress))
    text = "\rPercent: [{}] {:.3f}% {}".format( "#"*block + "-"*(10-block), progress*100, process)
    sys.stdout.write(text)
    sys.stdout.flush()

start_time = time.time()
missing_indices = []
for i in range(len(ra)):
    try:
        radec = driver.find_element_by_name('RA')
        driver.execute_script("arguments[0].value=arguments[1]",radec,"{}\t{}".format(ra[i],dec[i]))
        fits_file = driver.find_elements_by_name('ImageType')[2]
        fits_file.click()
        submit = driver.find_element_by_name('.submit')
        submit.click()
        #time.sleep(1)
    except NoSuchElementException:
        # Exception incase the input RA and dec are out of range of the database
        # Resets the webpage and continues loop through remaining values
        missing_indices.append(i)
        driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")
        time.sleep(3)
        imagesize = driver.find_element_by_name('ImageSize')
        driver.execute_script("arguments[0].value=arguments[1]", imagesize, '{}'.format(size_arcmin))
        continue
    update_progress('downloading images', (i+1)/len(ra))
    
print()
print('{} images downloaded in {:.3f} seconds'.format(len(ra), time.time()-start_time))
print(missing_indices)

Percent: [##########] 100.000% downloading images
100 images downloaded in 112.487 seconds
[]


In [8]:
# Compare length of ra,dec array with number of downloaded images

filepath = '../../Data/Garon 2017/matched.tsv'
data = np.array(pd.read_csv(filepath, sep='|', header=None, skiprows=3))
print(data.shape)

download_path = r'C:\Users\JakeM\Google Drive\University of Birmingham\Year 4\Project\Data\Garon 2017\FIRST-matched'
print(len(os.listdir(download_path)))

(3723, 10)
3722


In [146]:
filepath = '../../Data/Proctor 2011/asu.tsv'
data = pd.read_csv(filepath, sep='|', header=None, skiprows=3)
data = np.array(data)

ra, dec = data[:,1], data[:,2]

# Seperate into tables 1-14 for each morph type
table_ind = np.argwhere(data[:,8] != np.roll(data[:,8], 1)).flatten()
#t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14
tables = np.split(data, table_ind[1:])
ra, dec = np.split(ra, table_ind[1:]), np.split(dec, table_ind[1:])

In [137]:
def update_progress(process, progress):
        # Simple progress bar for long loops (e.g. data loading/augmentation)
        block = int(round(10*progress))
        text = "\rPercent: [{}] {:.3f}% {}".format( "#"*block + "-"*(10-block), progress*100, process)
        sys.stdout.write(text)
        sys.stdout.flush()

def FIRST_download(download_path, ra, dec):
    # Download FITS image files from FIRST cutout server
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' : download_path}
    chrome_options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    # Get webpage that cutouts can be downloaded from
    driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")

    size_arcmin = 4.5
    imagesize = driver.find_element_by_name('ImageSize')
    driver.execute_script("arguments[0].value=arguments[1]",imagesize,'{}'.format(size_arcmin))

    start_time = time.time()
    for i in range(len(ra)):
        try:
            radec = driver.find_element_by_name('RA')
            driver.execute_script("arguments[0].value=arguments[1]",radec,"{}\t{}".format(ra[i],dec[i]))
            fits_file = driver.find_elements_by_name('ImageType')[2]
            fits_file.click()
            submit = driver.find_element_by_name('.submit')
            submit.click()
            #time.sleep(1)
        except NoSuchElementException:
            # Exception incase the input RA and dec are out of range of the database
            # Resets the webpage and continues loop through remaining values
            print ('Could not find element {}'.format(i))
            driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")
            time.sleep(3)
            imagesize = driver.find_element_by_name('ImageSize')
            driver.execute_script("arguments[0].value=arguments[1]",imagesize,'2.5')
            continue
        update_progress('downloading images', (i+1)/len(ra))

    print('{} images downloaded in {:.3f} seconds'.format(len(ra), time.time()-start_time))

In [138]:
dir_path = r'C:\Users\JakeM\Google Drive\University of Birmingham\Year 4\Project\Data\Proctor 2011'
folders = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path,d))]

for i, (ra_, dec_, folder) in enumerate(zip(ra, dec, folders)):
    download_path = os.path.join(dir_path, folder)
    update_progress('downloading to '+folder, (i+1)/len(folders))
    FIRST_download(download_path, ra_, dec_)

Percent: [##########] 100.000% downloading images412 images downloaded in 336.514 seconds
Percent: [##########] 100.000% downloading imagesAT, NAT147 images downloaded in 115.857 seconds
Percent: [##########] 100.000% downloading images CJ45 images downloaded in 189.204 seconds
Percent: [##########] 100.000% downloading images SZ33 images downloaded in 25.965 seconds
Percent: [##########] 100.000% downloading images GRS65 images downloaded in 51.225 seconds
Percent: [##########] 100.000% downloading images tri82 images downloaded in 65.664 seconds
Percent: [##########] 100.000% downloading images QUAD28 images downloaded in 22.871 seconds
Percent: [##########] 100.000% downloading imagesW156 images downloaded in 127.700 seconds
Percent: [##########] 100.000% downloading imagesTB242 images downloaded in 201.600 seconds
Percent: [##########] 100.000% downloading imagesB103 images downloaded in 83.864 seconds
Percent: [##########] 100.000% downloading imagesRING400 images downloaded in 32

In [154]:
for f, ra_ in zip(folders, ra):
    print(len(os.listdir(os.path.join(dir_path,f))), f, '\t\t', len(ra_))

412 1 - WAT, NAT 		 412
103 10 - CJ 		 147
400 11 - SZ 		 45
48 12 - GRS 		 33
52 13 - tri 		 65
119 14 - QUAD 		 82
147 2 - W 		 28
45 3 - TB 		 156
33 4 - B 		 242
65 5 - RING 		 103
82 6 - Ring-lobe 		 400
28 7 - HYMOR 		 48
156 8 - X 		 52
242 9 - DD 		 119
